<a href="https://colab.research.google.com/github/Thiago-Reis-Porto/treinamento-h2ia/blob/main/damas_mimimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-------------------------------------------------------------------------------
# Play checkers against AI with minmax
#-------------------------------------------------------------------------------
game()
#-------------------------------------------------------------------------------

In [200]:
#-------------------------------------------------------------------------------
# Initialize pieces with board positions
#-------------------------------------------------------------------------------
board_size = 8
def init_pieces(rows):
  player = []
  ai = []
  
  #- Fomat : (x,y) = position, checker = bool / True if checker 
  #-((x,y), checker)
  for i in range(rows):
    for j in range(board_size):
      if not i%2 and not j%2:
        player.append(((i,j), False))
        ai.append(((board_size-1-i, j + 1), False))
      elif i%2 and j%2:
        player.append(((i,j),False))
        ai.append(((board_size-1-i, j - 1), False))

  return player, ai
#-------------------------------------------------------------------------------  

#-------------------------------------------------------------------------------  
# Check if colision with owned pieces or out of board
#-------------------------------------------------------------------------------  
def valid_pos(pos, is_player, player_pieces, AI_pieces):
  y,x = pos
  if x >= board_size or y >= board_size:
    return False
  
  if x < 0 or y < 0:
    return False
  
  if is_player:
    for i in player_pieces:
      if i[0] == pos: return False
  else:
    for i in AI_pieces:
      if i[0] == pos: return False

  return True
#-------------------------------------------------------------------------------  

#-------------------------------------------------------------------------------  
# Check if colision with enemy piece
#-------------------------------------------------------------------------------  
def enemy_colision(pos, is_player, player_pieces, AI_pieces):
  if is_player:
    for i in AI_pieces:
      if i[0] == pos: return True
  else:
    for i in player_pieces:
      if i[0] == pos: return True
  
  return False
#-------------------------------------------------------------------------------  
 
#-------------------------------------------------------------------------------  
# Check if position is empty
#-------------------------------------------------------------------------------
def is_empty(pos, player_pieces, AI_pieces, is_player): 
  if valid_pos(pos, is_player, player_pieces, AI_pieces):
    if not enemy_colision(pos, is_player, player_pieces, AI_pieces):
      return True
  return False 
#-------------------------------------------------------------------------------  

#---------------------------------------------------------------------------------------------------------------------------
# Get valid moves for the piece
#---------------------------------------------------------------------------------------------------------------------------
def get_moves(piece, is_player, player_pieces, AI_pieces):
  pos,checker = piece
  y,x = pos
  moves = []
  atacks = []
  b_jump = [0]
  
  #foward 
  direction_move_handler(pos, checker, 1, x, y, atacks, moves, [], 0, b_jump, is_player, True, player_pieces, AI_pieces)

  #backward
  direction_move_handler(pos, checker, -1, x, y, atacks, moves, [], 0, b_jump, is_player, False, player_pieces, AI_pieces)

  if atacks:
    return True, atacks
  else:
    return False, moves
#-----------------------------------------------------------------------------------------------------------------------------  

#------------------------------------------------------------------------------------------------------------------------------
# Aux function to check moviments/atacks on diagonal positions
#------------------------------------------------------------------------------------------------------------------------------
def direction_move_handler(pos, checker, m, x, y, atacks, moves, pieces_removed, t_jump, b_jump, is_player, foward, player_pieces, AI_pieces):
  if not is_player: m *= -1
  n = -1
  
  for i in range(2):
    new_pos = (y + m, x + n)

    if new_pos == pos: continue #only matter in atacks
    
    if valid_pos(new_pos, is_player, player_pieces, AI_pieces):
      if enemy_colision(new_pos, is_player, player_pieces, AI_pieces):
        #atack_handler 
        atack_handler(new_pos, m, n, is_player, atacks, pieces_removed, t_jump+1, b_jump, player_pieces, AI_pieces)
    
      elif foward:
        if not atacks:
          moves.append(new_pos)
      elif not atacks and checker:
         moves.append(new_pos)
    n*=-1
#----------------------------------------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------------------------------------
# return the biggest atack or atacks
#----------------------------------------------------------------------------------------------------------------------
# Paramameters:
#   pos - position, m\n - directios, b_jump - size of biggest atack so far
#   is_player - true if player turn, atacks list holding the atacks
#   atack format - (new postition, enemy pieces to remove)
#----------------------------------------------------------------------------------------------------------------------
def atack_handler(pos, y_dir, x_dir, is_player, atacks, pieces_removed, this_jump, b_jump, player_pieces, AI_pieces):

  y,x = pos
  cy = y+y_dir
  cx = x+x_dir
  check_pos = (cy, cx)
  
  if is_empty(check_pos, player_pieces, AI_pieces, is_player):
    
    pr = [*pieces_removed]
    pr.append(pos)
    if this_jump > b_jump[0]:
      b_jump[0]=this_jump
      atacks.clear()
      atacks.append((check_pos, pr))
    elif this_jump == b_jump[0]:
      atacks.append((check_pos, pr))
  else: return
  
  # m\n idicate de direction of the move
  # Check for Foward atacks
  direction_move_handler(pos, False, 1, cx, cy, atacks, [], pr, this_jump, b_jump, is_player, True, player_pieces, AI_pieces)
  
  # Backward atacks
  direction_move_handler(pos, False, -1, cx, cy, atacks, [], pr,this_jump, b_jump, is_player, False, player_pieces, AI_pieces)
  
  #----------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------
# Generate an new state based on action received
#-------------------------------------------------------------------------------------
def generate_state(action, is_atack, piece, player_piece, player_pieces, AI_pieces):
  
  AI = [*AI_pieces]
  player = [*player_pieces]

  if not is_atack:
    move = (action, piece[1])
    if player_piece:
      if move[0][0] == 7: move = (action, True)
      player.remove(piece); player.append(move)
    else:
      if move[0][0] == 0: move = (action, True) 
      AI.remove(piece); AI.append(move)
  
  else:
    move = (action[0], piece[1]) 
    to_remove = action[1]
    if player_piece:
      if move[0][0] == 7: move = (action[0], True)
      player.remove(piece); player.append(move)
      while to_remove:
        aux = get_piece(to_remove.pop(), AI)
        AI.remove(aux)
    else:
      if move[0][0] == 0: move = (action[0], True)
      AI.remove(piece); AI.append(move)
      while to_remove:
        aux = get_piece(to_remove.pop(), player)
        player.remove(aux)
  
  return player, AI
#-------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------
# Get piece by position
#-------------------------------------------------------------------------------------
def get_piece(pos, pieces):
  for i in pieces:
    if i[0] == pos:
      return i
  return None
#-------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------
# Main game function
#-------------------------------------------------------------------------------------
no_atack = True
no_atacks = 0
exit_game = False
def game():
  pieces_rows = 2
  # Initialization--------------------------------------------
  player_pieces, AI_pieces = init_pieces(pieces_rows)
  createCheckersBoard(8, player_pieces, AI_pieces, True)
  # ----------------------------------------------------------
 
  game_loop(player_pieces, AI_pieces)
#--------------------------------------------------------------


from IPython.display import clear_output 
#--------------------------------------------------------------
# Main game Loop
#--------------------------------------------------------------
def game_loop(player_pieces, AI_pieces):
  global no_atack, no_atacks
  no_atack = True
  #player turn
  player_pieces, AI_pieces = player_turn(player_pieces, AI_pieces)
  #AI turn
  player_pieces, AI_pieces = minmax(player_pieces, AI_pieces)

  if no_atack: no_atacks += 1
  else: no_atacks = 0
  
  clear_output()
  createCheckersBoard(8, player_pieces, AI_pieces, True)
  
  if not end_game(player_pieces, AI_pieces):
    game_loop(player_pieces, AI_pieces)
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Check ending conditions
#-------------------------------------------------------------------------------
def end_game(player, ai):
  if no_atacks >= 20: return True
  if not player or not ai: return True
  
  return False
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------
# Player turn handler, returs the result state
#-------------------------------------------------------------------------------------
def player_turn(player_pieces, AI_pieces):
  global no_atack
  # Pick player choices----------------------------------
  p = get_avaliable_moves(player_pieces, AI_pieces, True)
  if not p: return player_pieces, AI_pieces
  choice = player_select_avaliable_piece(p) - 1 
  piece_move = p[choice]
  choice = player_select_move(piece_move) - 1
  #------------------------------------------------------
  piece = piece_move[0]
  action = piece_move[1][1][choice]
  is_atack = piece_move[1][0]
  player_piece = True
  if is_atack: no_atack = False

  return generate_state(action, is_atack, piece, player_piece, player_pieces, AI_pieces)
#-------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------
# Return a list of tuples with piece and avaliable actions
#-------------------------------------------------------------------------------------
def get_avaliable_moves(player, ai, is_player):
  pieces_action = []
  biggest_atack = 0
  if is_player:
    for piece in player:
      moves = get_moves(piece, is_player, player, ai)
      if moves[0]:
        if len(moves[1][0][1]) == biggest_atack:
          atacks.append((piece, moves))
        if len(moves[1][0][1]) > biggest_atack:
          atacks = [(piece, moves)]
          biggest_atack = len(moves[1][0][1])
      elif moves[1]: pieces_action.append((piece, moves))
  else: 
    for piece in ai:
      moves = get_moves(piece, is_player, player, ai)
      if moves[0]:
        if len(moves[1][0][1]) > biggest_atack:
          atacks = [(piece, moves)]
          biggest_atack = len(moves[1][0][1])
        if len(moves[1][0][1]) == biggest_atack:
          atacks.append((piece, moves))
      elif moves[1]: pieces_action.append((piece, moves))
  if biggest_atack: pieces_action = atacks
  return pieces_action
#-------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------
# Pick the index of piece selected
#-------------------------------------------------------------------------------------
def player_select_avaliable_piece(pieces_action):
  i = 0
  while i<=0 or i>len(pieces_action):
    print("Select a piece to move:")
    for i in range(len(pieces_action)):
      print(i+1, pieces_action[i][0][0])
    i = int(input())
  return i
#-------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------
# Pick the index of the move selected
#-------------------------------------------------------------------------------------
def player_select_move(piece_move):
  moves = piece_move[1][1]
  i=0
  while i<=0 or i>len(moves):
    print("Select a moviment:")
    for i in range(len(moves)):
      print(i+1, moves[i])
    i = int(input())
  return i
#-------------------------------------------------------------------------------------
#-----------------------------------------------------------------------
# Evaluate the board based on some heuristic
#-----------------------------------------------------------------------
def evaluate(player_pieces, AI_pieces):
  value = 0

  #if not player_pieces == 0: return 16
  #if not AI_pieces == 0: return -16

  for i in player_pieces:
    if not i[1]: value -=1
    else: value -=2
  
  for i in AI_pieces:
    if not i[1]: value +=1
    else: value +=2
  
  return value
#-----------------------------------------------------------------------
#----------------------------------------------------------------------------------
# Minmax function for IA plays
#----------------------------------------------------------------------------------
# alfa - only increases -> biggest so far
# beta - only decreases -> smales so far
#----------------------------------------------------------------------------------
max_level = 5
def minmax(player_pieces, AI_pieces):
  
  pieces = get_avaliable_moves(player_pieces, AI_pieces, False)
  if not pieces: return player_pieces, AI_pieces
  
  alfa = float('-inf')
  biggest = None
  b_atack = False
  
  for piece in pieces:
    atack = piece[1][0]
    moves = piece[1][1]

    for move in moves:
      p, ai = generate_state(move, atack, piece[0], False, player_pieces, AI_pieces)
      v = return_min(p, ai, 1, alfa)
      
      if v == None: continue
      
      if v > alfa:
        alfa = v
        biggest = p, ai
        b_atack = atack
  
  global no_atack
  if b_atack: no_atack = False
  return biggest
  #----------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# return max value of path with pruning
#----------------------------------------------------------------------------------
def return_max(player_pieces, AI_pieces, level, beta):
  
  if not player_pieces : return 16
  if not AI_pieces : return -16
  if level == max_level: return evaluate(player_pieces, AI_pieces)

  pieces = get_avaliable_moves(player_pieces, AI_pieces, False)
  if not pieces: return evaluate(player_pieces, AI_pieces)
  
  alfa = float('-inf')
 
  for piece in pieces:
    atack = piece[1][0]
    moves = piece[1][1]

    for move in moves:
      p, ai = generate_state(move, atack, piece[0], False, player_pieces, AI_pieces)
      v = return_min(p, ai, level+1, alfa)
     
      if v == None: continue
      if v > alfa: alfa = v
      if alfa > beta : return None
       
  return alfa
  #----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
# Return min value of path with pruning
#----------------------------------------------------------------------------------
def return_min(player_pieces, AI_pieces, level, alfa):
  
  if not player_pieces : return 16
  if not AI_pieces : return -16
  if level == max_level: return evaluate(player_pieces, AI_pieces)

  pieces = get_avaliable_moves(player_pieces, AI_pieces, True)
  if not pieces: return evaluate(player_pieces, AI_pieces)
  
  beta = float('inf')

  for piece in pieces: 
    atack = piece[1][0]
    moves = piece[1][1]

    for move in moves:
      p, ai = generate_state(move, atack, piece[0], True, player_pieces, AI_pieces)
      v = return_max(p, ai, level+1, beta)
     
      if v == None: continue
      if v < beta: beta = v
      if beta < alfa : return None
      
  return beta
#----------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------
# Draw the board using matplot
#-----------------------------------------------------------------------------------------------------

# matplot board code based on: https://gist.github.com/Andrewnetwork/a59f60daa019d99529f7622bef50a105
# TODO : Change the board instead to drawn new one each turn

import matplotlib.pyplot as plt
import matplotlib.patches as patches

#-------------------------------------------------------------------------------
# Canvas initialization
#-------------------------------------------------------------------------------  
my_dpi = 60
def create_canvas(width,height):
    fig,ax = plt.subplots(1)
    fig.dpi= my_dpi
    fig.set_size_inches(width,height)
    ax.set_xticks([])
    ax.set_yticks([])
    return fig,ax

#-------------------------------------------------------------------------------
# Create a square for the board
#-------------------------------------------------------------------------------      
#  wh: (width,height) | xy: (x,y) | cwch: (canvasWidth,canvasHeight)
def create_rectangle( wh, xy, cwch, color="black"):
    return patches.Rectangle((xy[0]/cwch[0],xy[1]/cwch[1]),wh[0]/cwch[0],wh[1]/cwch[1],color=color)
#-------------------------------------------------------------------------------  

#-------------------------------------------------------------------------------  
# Create circle for a piece
#-------------------------------------------------------------------------------  
def create_circle( xy, cwch, radius, color):
  return patches.Circle((xy[0]/cwch[0], xy[1]/cwch[1]), radius, color=color)
#-------------------------------------------------------------------------------  

#-------------------------------------------------------------------------------  
# Draws the board
#-------------------------------------------------------------------------------  
def createCheckersBoard(n, player_pieces, AI_pieces,captionLoc=False):
    assert n % 2 == 0, 'n is not even'
    canvasSize = (n,n)
    fig,ax = create_canvas(canvasSize[0],canvasSize[1])
    color = "black"

    for row in range(n):
        for col in range(n):
          
            rect = create_rectangle((1,1),(col,row),canvasSize,color)
            
            ax.add_patch(rect)
            if captionLoc:
                locStr = "({0},{1})".format(col,row)
                ax.text((row+0.23)/n,(col+0.35)/n,locStr,fontsize=12,color="white")
                
            # Color Oscillation-------------------------------------------------
            if color == "black" and col != n-1:
                color = "red"
            elif col != n-1:
                color = "black"
            #-------------------------------------------------------------------

  # Put pieces on board---------------------------------------------------------      
    for piece in player_pieces:
      pos,checker = piece
      y,x = pos
      if not checker: color = (1, 0.392, 0)
      else: color = (1, 0.235, 0)
      cir = create_circle( (x + 0.5, y + 0.5), canvasSize, 0.05, color)
      ax.add_patch(cir) 

    for piece in AI_pieces: 
      pos,checker = piece
      y,x = pos
      if not checker: color = (0, 0.470, 0)
      else: color = (0, 0.235, 0)
      cir = create_circle( (x + 0.5, y + 0.5), canvasSize, 0.05, color)
      ax.add_patch(cir) 
  
    plt.show()
#-------------------------------------------------------------------------------  